# Bipolar Junction Transistor (BJT)

## Current-Controlled Amplification

The BJT is a three-terminal device where a small base current controls a larger collector current. It remains important for analog circuits and high-frequency applications.

**Learning Objectives:**
- Understand BJT operation (NPN and PNP)
- Simulate common-emitter characteristics
- Generate Gummel plots
- Extract current gain (beta)

---

## 1. BJT Physics

### 1.1 Structure (NPN)

```
   Emitter (N+)  |  Base (P)  |  Collector (N)
   ============  |  ========  |  =============
   High doping   |  Thin,     |  Low doping
                 |  moderate  |
                 |  doping    |
```

### 1.2 Key Parameters

- **Current gain** ($\beta$ or $h_{FE}$):
  $$\beta = \frac{I_C}{I_B}$$

- **Collector current**:
  $$I_C = I_S \exp\left(\frac{V_{BE}}{V_T}\right)$$

- **Base current**:
  $$I_B = \frac{I_C}{\beta}$$

- **Early voltage** ($V_A$): Models output resistance
  $$I_C = I_S \exp\left(\frac{V_{BE}}{V_T}\right)\left(1 + \frac{V_{CE}}{V_A}\right)$$

In [ ]:
import numpy as np
import matplotlib.pyplot as plt
from nanohubpadre import create_bjt

# Physical constants
q = 1.6e-19
kT = 0.0259  # Thermal voltage at 300K

print("BJT Parameters:")
print("="*40)
print(f"Thermal voltage VT = {kT*1000:.1f} mV")
print(f"Typical beta (hFE) = 50-300")
print(f"Typical Early voltage VA = 50-200 V")

---

## 2. Creating a BJT Simulation

In [ ]:
# Create NPN transistor
sim_npn = create_bjt(
    # Geometry
    emitter_width=1.0,      # Emitter region width (um)
    base_width=0.3,         # Base region width (um) - thin!
    collector_width=2.0,    # Collector region width (um)
    device_depth=1.0,
    
    # Doping
    emitter_doping=1e20,    # N+ emitter
    base_doping=1e17,       # P base
    collector_doping=1e16,  # N collector
    device_type='npn',
    
    # Models
    temperature=300,
    srh=True,
    auger=True,
    bgn=True,
    
    # Output
    log_bands_eq=True
)

print("NPN BJT Configuration:")
print("="*40)
print("Emitter (N+): 1e20 cm^-3")
print("Base (P): 1e17 cm^-3, 0.3 um wide")
print("Collector (N): 1e16 cm^-3")

In [ ]:
# View generated deck
print("PADRE Input Deck:")
print("="*60)
print(sim_npn.generate_deck())

---

## 3. Common-Emitter Output Characteristics (Ic vs Vce)

In [ ]:
# Common-emitter output characteristics
sim_ce = create_bjt(
    base_width=0.3,
    emitter_doping=1e20,
    base_doping=1e17,
    collector_doping=1e16,
    device_type='npn',
    
    # Enable I-V logging
    log_iv=True,
    iv_file="ic_vce",
    
    # Output characteristic: sweep Vce at fixed Vbe
    vbe=0.7,                      # Base-emitter voltage
    vce_sweep=(0.0, 3.0, 0.1)     # Vce: 0 to 3V
)

print("Common-Emitter Output Characteristic")
print("="*40)
print("Vbe = 0.7V (fixed)")
print("Vce sweep: 0 to 3V")

In [ ]:
# Theoretical Ic-Vce curves
Vce = np.linspace(0.1, 5, 100)
Is = 1e-15  # Saturation current
VA = 100    # Early voltage
Vbe_values = [0.65, 0.70, 0.75, 0.80]

plt.figure(figsize=(10, 6))

for Vbe in Vbe_values:
    Ic = Is * np.exp(Vbe / kT) * (1 + Vce / VA)
    plt.plot(Vce, Ic * 1e3, linewidth=2, label=f'Vbe = {Vbe}V')

plt.xlabel('Collector-Emitter Voltage Vce (V)', fontsize=12)
plt.ylabel('Collector Current Ic (mA)', fontsize=12)
plt.title('BJT Common-Emitter Output Characteristics', fontsize=14)
plt.legend()
plt.grid(True, alpha=0.3)
plt.xlim(0, 5)
plt.ylim(0, None)

# Add Early voltage extrapolation
plt.annotate('Early voltage\nVA ~ -100V', xy=(-0.5, 0), fontsize=10,
             xytext=(1, 0.5), arrowprops=dict(arrowstyle='->'))

plt.tight_layout()
plt.show()

---

## 4. Gummel Plot (Ic, Ib vs Vbe)

In [ ]:
# Gummel plot simulation
sim_gummel = create_bjt(
    base_width=0.3,
    emitter_doping=1e20,
    base_doping=1e17,
    collector_doping=1e16,
    device_type='npn',
    
    # Enable I-V logging
    log_iv=True,
    iv_file="gummel",
    
    # Gummel plot: sweep Vbe at fixed Vce
    gummel_sweep=(0.0, 0.9, 0.02),  # Vbe: 0 to 0.9V
    gummel_vce=2.0                   # Fixed Vce = 2V
)

print("Gummel Plot Simulation")
print("="*40)
print("Vbe sweep: 0 to 0.9V")
print("Vce = 2.0V (fixed)")

In [ ]:
# Theoretical Gummel plot
Vbe = np.linspace(0.3, 0.9, 100)
Is = 1e-15
beta = 100

Ic = Is * np.exp(Vbe / kT)
Ib = Ic / beta

plt.figure(figsize=(10, 6))

plt.semilogy(Vbe, Ic, 'b-', linewidth=2, label='Ic')
plt.semilogy(Vbe, Ib, 'r-', linewidth=2, label='Ib')

plt.xlabel('Base-Emitter Voltage Vbe (V)', fontsize=12)
plt.ylabel('Current (A)', fontsize=12)
plt.title('Gummel Plot', fontsize=14)
plt.legend()
plt.grid(True, alpha=0.3)
plt.xlim(0.3, 0.9)

# Add slope annotation
plt.annotate('Slope = q/kT\n(60 mV/decade)', xy=(0.5, 1e-9), fontsize=10)

# Add beta annotation
plt.annotate(f'beta = Ic/Ib = {beta}', xy=(0.7, 1e-6), fontsize=10)

plt.tight_layout()
plt.show()

---

## 5. Current Gain (Beta) Analysis

In [ ]:
# Beta vs collector current
Ic = np.logspace(-9, -2, 100)

# Beta varies with current (simplified model)
# Low current: recombination in E-B depletion region
# High current: high-injection effects
beta_max = 150
Ic_peak = 1e-4
beta = beta_max / (1 + (Ic/Ic_peak)**0.5 + (Ic_peak/Ic)**0.3)

plt.figure(figsize=(10, 5))

plt.semilogx(Ic * 1e3, beta, 'b-', linewidth=2)
plt.xlabel('Collector Current Ic (mA)', fontsize=12)
plt.ylabel('Current Gain beta', fontsize=12)
plt.title('Beta vs Collector Current', fontsize=14)
plt.grid(True, alpha=0.3)
plt.ylim(0, 200)

# Annotate regions
plt.annotate('Low Ic:\nRecombination\nlimits beta', xy=(1e-6, 50), fontsize=10)
plt.annotate('High Ic:\nHigh injection\nlimits beta', xy=(1, 50), fontsize=10)

plt.tight_layout()
plt.show()

---

## 6. Base Width Effect

In [ ]:
# Compare different base widths
base_widths = [0.2, 0.3, 0.5, 1.0]  # microns

print("Base Width Comparison:")
print("="*50)

for Wb in base_widths:
    sim = create_bjt(
        base_width=Wb,
        emitter_doping=1e20,
        base_doping=1e17,
        collector_doping=1e16,
        device_type='npn',
        log_iv=True,
        gummel_sweep=(0.0, 0.8, 0.02)
    )
    
    # Theoretical beta (simplified)
    # beta ~ Dn*Ln/(Dp*Wb) for uniform doping
    print(f"Wb = {Wb} um: Thinner base -> Higher beta")

---

## 7. NPN vs PNP

In [ ]:
# NPN transistor
sim_npn = create_bjt(
    device_type='npn',
    log_iv=True,
    vbe=0.7,
    vce_sweep=(0.0, 3.0, 0.1)
)

# PNP transistor
sim_pnp = create_bjt(
    device_type='pnp',
    log_iv=True,
    vbe=-0.7,  # Negative for PNP
    vce_sweep=(0.0, -3.0, -0.1)  # Negative for PNP
)

print("NPN vs PNP:")
print("="*50)
print("\nNPN: Electrons flow E->C, conventional current C->E")
print("  Vbe > 0, Vce > 0 for active mode")
print("\nPNP: Holes flow E->C, conventional current E->C")
print("  Vbe < 0, Vce < 0 for active mode")
print("  Generally lower beta (hole mobility < electron mobility)")

---

## 8. Complete Simulation Example

In [ ]:
# Complete BJT characterization
sim_complete = create_bjt(
    # Geometry
    emitter_width=1.0,
    base_width=0.3,
    collector_width=2.0,
    device_depth=1.0,
    
    # Mesh
    nx=150,
    ny=30,
    
    # Doping
    emitter_doping=1e20,
    base_doping=1e17,
    collector_doping=1e16,
    device_type='npn',
    
    # Models
    temperature=300,
    srh=True,
    auger=True,
    bgn=True,
    conmob=True,
    fldmob=True,
    
    # Output
    log_iv=True,
    iv_file="bjt_complete",
    log_bands_eq=True,
    
    # Gummel plot for characterization
    gummel_sweep=(0.0, 0.85, 0.01),
    gummel_vce=2.0
)

print("Complete BJT Simulation")
print("="*50)
print(sim_complete.generate_deck())

---

## Summary

In this notebook, you learned:

1. **BJT Structure**: Emitter, Base, Collector regions
2. **Output Characteristics**: Ic vs Vce at various Vbe
3. **Gummel Plot**: Ic and Ib vs Vbe (exponential)
4. **Current Gain**: Beta = Ic/Ib, varies with current
5. **Design Parameters**: Base width critical for beta

**Key Equations:**
- $I_C = I_S e^{V_{BE}/V_T}$
- $\beta = I_C / I_B$

**Next**: [06 - Solar Cell](06_Solar_Cell.ipynb) - Photovoltaic devices